In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [4]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda:6')
else:
    DEVICE = torch.device("cpu")

print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.10.2+cu102  Device: cuda:6


In [5]:
BATCHSIZE = 32
EPOCHS = 10

In [8]:
train_datasets = datasets.CIFAR10(root='../data/CIFAR_10', train=True, download=True, transform=transforms.Compose([transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]))
test_datasets = datasets.CIFAR10(root='../data/CIFAR_10', train=False, transform=transforms.Compose([transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]))
train_loader = torch.utils.data.DataLoader(dataset= train_datasets, batch_size=BATCHSIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset= test_datasets, batch_size=BATCHSIZE, shuffle=False)

Files already downloaded and verified


In [36]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride = 1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size= 3, stride = stride,padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size= 3, stride = 1,padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(nn.Conv2d(in_planes, planes, kernel_size = 1, stride = stride, bias= False), nn.BatchNorm2d(planes))
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [37]:
class ResNet(nn.Module):
    def __init__(self, num_Classes = 10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride = 1)
        self.layer2 = self._make_layer(32, 2, stride = 2)
        self.layer3 = self._make_layer(64, 2, stride = 2)
        self.linear = nn.Linear(64, num_Classes)
    
    def _make_layer(self,planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [38]:
model = ResNet().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [39]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [40]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [41]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim=True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()

    test_loss /= (len(test_loader.dataset)/BATCHSIZE)
    test_accuracy = 100 * correct / len(test_loader.dataset)

    return test_loss, test_accuracy

In [42]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.435867
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.623470
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.612515
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.051703
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.695855
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.090916
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.079330
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.202735

[EPOCH: 1], 	Test Loss: 1.0356, 	Test Accuracy: 63.24 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 0.881009
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 0.810281
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 0.664034
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 0.850634
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 0.773943
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 0.899879
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 0.712756
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 0.744185

[EPOCH: 2], 	Test Loss: 0.8723, 	Test Accuracy: 69.10 % 